In [5]:
CONFIGS = {
    'data_path': '../data/',
}

In [59]:
import numpy as np
import pandas as pd

import pickle

import datetime

In [6]:
with open(CONFIGS['data_path']+'train_prob1.pkl', 'rb') as f:
    train_origin = pickle.load(f)
    
with open(CONFIGS['data_path']+'test_prob1.pkl', 'rb') as f:
    test_origin = pickle.load(f)

In [53]:
train = train_origin.copy()
test = test_origin.copy()

In [54]:
def status_to_int(data):
    new_data = data.copy()
    new_data['status'] = new_data['status'].apply(lambda x: int(x[-1]))
    return new_data

def code_count_in_code(data, train):
    oc = train.groupby(['origin_code'], as_index=False)[['dt']].count()\
        .rename(columns={'origin_code': 'code', 'dt': 'oc'})
    dc = train.groupby(['destination_code'], as_index=False)[['dt']].count()\
        .rename(columns={'destination_code': 'code', 'dt': 'dc'})

    code_count = pd.merge(oc, dc, how='outer', on='code').fillna(0)

    merge_oc = pd.merge(
        data, code_count.rename(columns={'code': 'origin_code'}),
        how='left', on='origin_code'
    ).rename(columns={'oc': 'oc_in_oc', 'dc': 'oc_in_dc'})

    merge_dc = pd.merge(
        merge_oc, code_count.rename(columns={'code': 'destination_code'}),
        how='left', on='destination_code'
    ).rename(columns={'oc': 'dc_in_oc', 'dc': 'dc_in_dc'})
    
    return merge_dc.fillna(0)

def mk_time_data(data):
    new_data = data.copy()
    new_data['dt'] = new_data['dt'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
    
    new_data['day'] = new_data['dt'].apply(lambda x: x.day)
    
    new_data['hour'] = new_data['dt'].apply(lambda x: x.hour)
    new_data['cos_hour'] = np.cos(2*np.pi*(new_data['hour']/24))
    new_data['sin_hour'] = np.sin(2*np.pi*(new_data['hour']/24))

    new_data['minute'] = new_data['dt'].apply(lambda x: x.minute)
    new_data['cos_minute'] = np.cos(2*np.pi*(new_data['minute']/60))
    new_data['sin_minute'] = np.sin(2*np.pi*(new_data['minute']/60))

    new_data['ts'] = new_data['hour']*6 + new_data['minute']//10
    new_data['cos_ts'] = np.cos(2*np.pi*(new_data['ts']/(24*6)))
    new_data['sin_ts'] = np.sin(2*np.pi*(new_data['ts']/(24*6)))

    new_data['weekday'] = new_data['dt'].apply(lambda x: x.weekday())
    new_data['cos_weekday'] = np.cos(2*np.pi*(new_data['weekday']/7))
    new_data['sin_weekday'] = np.sin(2*np.pi*(new_data['weekday']/7))

    new_data['weekts'] = new_data['weekday']*144 + new_data['ts']
    new_data['cos_weekts'] = np.cos(2*np.pi*(new_data['weekts']/(144*7)))
    new_data['sin_weekts'] = np.sin(2*np.pi*(new_data['weekts']/(144*7)))

    new_data['dt'] = new_data['dt'].apply(lambda x: str(x))

    new_data['is_weekend'] = 0
    new_data.loc[
        (new_data['weekday'] == 5) | (new_data['weekday'] == 6),
        'is_weekend'
    ] = 1

    new_data['is_holiday'] = 0
    new_data.loc[
        (new_data['day'] == 5) | (new_data['day'] == 19),
        'is_holiday'
    ] = 1

    new_data['is_before_holiday'] = 0
    new_data.loc[
        (new_data['day'] == 4) | (new_data['day'] == 18),
        'is_before_holiday'
    ] = 1

    return new_data

In [55]:
train, test = status_to_int(train), status_to_int(test)
train, test = code_count_in_code(train, train), code_count_in_code(test, train)
train, test = mk_time_data(train), mk_time_data(test)